In [1]:
import numpy as np
import cupy as cp

## Fast Fourier Transform (FFT)

CuPy offers the fft -library to make fast Fourier transforms.

Lets test it and create couple of arrays.

In [2]:
a_cpu = np.array([1, 2, 3, 4, 5])
a_gpu = cp.array([1, 2, 3, 4, 5])


Using the NumPys fft() function, calculate the Fourier Transform

In [3]:
fourier_cpu = np.fft.fft(a_cpu)

Using the CuPys fft() function, calculate the Fourier Transform

In [4]:
fourier_gpu = cp.fft.fft(a_gpu)

Print the results.

In [5]:
print(fourier_cpu)
print(fourier_gpu)

[15. +0.j         -2.5+3.4409548j  -2.5+0.81229924j -2.5-0.81229924j
 -2.5-3.4409548j ]
[15. +0.j         -2.5+3.4409548j  -2.5+0.81229924j -2.5-0.81229924j
 -2.5-3.4409548j ]


Seems about the same.

How about the speed difference?

Lets check with an array of million values.

In [6]:
a_cpu = np.random.randn(10**6)
a_gpu = cp.random.randn(10**6)

In [7]:
%%timeit
fourier_cpu = np.fft.fft(a_cpu)

42.8 ms ± 1.67 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
%%timeit
fourier_gpu = cp.fft.fft(a_gpu)

1.29 ms ± 2.38 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Seems like already quite a good performance improvement.

## Sparse matrices

To work with sparse matrices, CuPy offers the cupyx.scipy.sparse -library

In [9]:
import cupy as cp
import cupyx as cpx


Lets create a dense matrix.

In [10]:
arr = cp.array([[0, 0, 0, 0, 0, 1, 1, 0, 2],[0, 7, 0, 4, 0, 1, 1, 0, 2]], dtype=np.float64)
print(arr)

[[0. 0. 0. 0. 0. 1. 1. 0. 2.]
 [0. 7. 0. 4. 0. 1. 1. 0. 2.]]


We can now use the sparse library to convert it into the COO-format.

In [11]:
arr_coo = cpx.scipy.sparse.coo_matrix(arr)
print(arr_coo)

  (0, 5)	1.0
  (0, 6)	1.0
  (0, 8)	2.0
  (1, 1)	7.0
  (1, 3)	4.0
  (1, 5)	1.0
  (1, 6)	1.0
  (1, 8)	2.0


The sparse library allows us also to create random sparse matrices.

In [12]:
arr = cpx.scipy.sparse.rand(5, 5, density=0.3)
print(type(arr))
print(arr)

<class 'cupyx.scipy.sparse._coo.coo_matrix'>
  (0, 3)	0.20003036440584637
  (1, 2)	0.15719355677507568
  (3, 3)	0.21528667638850835
  (2, 3)	0.0295052730802024
  (3, 4)	0.20567788988132735
  (2, 2)	0.3201104800818669
  (2, 4)	0.5621865572034708


If we want to see it in the "normal" format. We can use the .todense() method.

In [13]:
arr.todense()

array([[0.        , 0.        , 0.        , 0.20003036, 0.        ],
       [0.        , 0.        , 0.15719356, 0.        , 0.        ],
       [0.        , 0.        , 0.32011048, 0.02950527, 0.56218656],
       [0.        , 0.        , 0.        , 0.21528668, 0.20567789],
       [0.        , 0.        , 0.        , 0.        , 0.        ]])

Want to see all the elements with non zero values? No problem. Use the .data property.

In [14]:
print(arr.data)

[0.20003036 0.15719356 0.21528668 0.02950527 0.20567789 0.32011048
 0.56218656]


It is also possible to convert the sparse matrix from one format to another. For example, to the csr-format by using the csr_matrix() -function.

In [15]:
arr_csr = cpx.scipy.sparse.csr_matrix(arr)
print(type(arr_csr))
print(arr_csr)

<class 'cupyx.scipy.sparse._csr.csr_matrix'>
  (0, 3)	0.20003036440584637
  (1, 2)	0.15719355677507568
  (2, 2)	0.3201104800818669
  (2, 3)	0.0295052730802024
  (2, 4)	0.5621865572034708
  (3, 3)	0.21528667638850835
  (3, 4)	0.20567788988132735


Note that the values are still printed in COO-format, since this is a bit more viewer-friendly way of doing it.

How about operations?

In [16]:
A = cp.array([[1, 2, 0], [0, 0, 3], [4, 0, 5]], dtype=np.float64)
A_csr = cpx.scipy.sparse.csr_matrix(A)

v = cp.array([1, 0, -1])

A_csr.dot(v)

array([ 1., -3., -1.])

## Random

For generation of random values, CuPy offers the random-library which is working on top of the NVidias cuRAND accelerated library.

In [17]:
import cupy as cp

# generate random integer from 0 to 9
random_number = cp.random.randint(0, 10)

print(random_number)

0


In [18]:
# generate random float-point number between 0 and 1
random_number = cp.random.rand()

print(random_number)

0.2599208077327239


In [19]:
# generate 1D array of 5 random integers between 0 and 9
integer_array = cp.random.randint(0, 10, 5)

print("1D Random Integer Array:\n",integer_array)

# generate 1D array of 5 random numbers between 0 and 1
float_array = cp.random.rand(5)

print("\n1D Random Float Array:\n",float_array)

# generate 2D array of shape (3, 4) with random integers
result = cp.random.randint(0, 10, (3,4))

print("\n2D Random Integer Array:\n",result)

1D Random Integer Array:
 [5 7 0 4 0]

1D Random Float Array:
 [0.40192731 0.71384671 0.62001214 0.2684872  0.64694475]

2D Random Integer Array:
 [[8 4 8 6]
 [5 5 1 9]
 [1 8 8 8]]


In [20]:
# create an array of integers from 1 to 5
array1 = cp.array([1, 2, 3, 4, 5])

# choose a random number from array1. CuPy specific, need to specify size.
random_choice = cp.random.choice(array1,size=1)

print(random_choice)

[3]
